In [1]:
import os
import numpy as np
import tensorflow as tf
from pathlib import Path
from tensorflow import keras
from vgg16_model import VGG16
from sklearn.model_selection import train_test_split

### Load data

In [2]:
# Define the data directory containing all images organized by class
data_dir = Path(r'Datasets/Faulty_solar_panel')

def load_data_from_directory(directory, img_size=(299, 299)):
    images = []
    labels = []
    class_names = []
    
    for class_idx, class_dir in enumerate(sorted(os.listdir(directory))):
        class_path = os.path.join(directory, class_dir)
        if not os.path.isdir(class_path):
            continue
        
        class_names.append(class_dir)
        
        for img_file in os.listdir(class_path):
            if img_file.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
                img_path = os.path.join(class_path, img_file)
                try:
                    img = keras.preprocessing.image.load_img(img_path, target_size=img_size)
                    img_array = keras.preprocessing.image.img_to_array(img)
                    images.append(img_array)
                    labels.append(class_idx)
                except Exception as e:
                    print(f"Error loading {img_path}: {e}")
    
    return np.array(images), np.array(labels), class_names


# Load all data from the Datasets directory
try:
    all_images, all_labels, class_names = load_data_from_directory(data_dir)
    print(f"Loaded all data: {all_images.shape}")
    print(f"Classes: {class_names}\n")
except Exception as e:
    print(f"Error loading data: {e}")
    breakpoint()

# Split data: 70% train, 15% validation, 15% test
train_images, temp_images, train_labels, temp_labels = train_test_split(
    all_images, all_labels, test_size=0.3, random_state=42, stratify=all_labels
)

validation_images, test_images, validation_labels, test_labels = train_test_split(
    temp_images, temp_labels, test_size=0.5, random_state=42, stratify=temp_labels
)

print(f"Train images shape: {train_images.shape}")
print(f"Validation images shape: {validation_images.shape}")
print(f"Test images shape: {test_images.shape}")

Loaded all data: (807, 299, 299, 3)
Classes: ['Bird-drop', 'Clean', 'Dusty', 'Electrical-damage', 'Physical-Damage', 'Snow-Covered']

Train images shape: (564, 299, 299, 3)
Validation images shape: (121, 299, 299, 3)
Test images shape: (122, 299, 299, 3)


In [3]:
# Create TensorFlow datasets from the pre-split data
print(f"Creating TensorFlow datasets...")
print(f"Training samples: {len(train_images)}")
print(f"Validation samples: {len(validation_images)}")
print(f"Test samples: {len(test_images)}\n")

# Building tensorflow datasets
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
validation_ds = tf.data.Dataset.from_tensor_slices((validation_images, validation_labels))
test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

Creating TensorFlow datasets...
Training samples: 564
Validation samples: 121
Test samples: 122



In [4]:
def process_images(image, label):
    # Normalize images to have a mean of 0 and standard deviation of 1
    image = tf.image.per_image_standardization(image)
    image = tf.image.resize(image, (299, 299))
    return image, label

train_ds_size = tf.data.experimental.cardinality(train_ds).numpy()
test_ds_size = tf.data.experimental.cardinality(test_ds).numpy()
validation_ds_size = tf.data.experimental.cardinality(validation_ds).numpy()

print(f"Dataset sizes:")
print(f"Training batches: {train_ds_size}")
print(f"Validation batches: {validation_ds_size}")
print(f"Test batches: {test_ds_size}\n")

train_ds = (train_ds
            .map(process_images)
            .shuffle(buffer_size=train_ds_size)
            .batch(batch_size=16, drop_remainder=True))
test_ds = (test_ds
           .map(process_images)
           .shuffle(buffer_size=test_ds_size)
           .batch(batch_size=16, drop_remainder=True))
validation_ds = (validation_ds
                 .map(process_images)
                 .shuffle(buffer_size=validation_ds_size)
                 .batch(batch_size=16, drop_remainder=True))

Dataset sizes:
Training batches: 564
Validation batches: 121
Test batches: 122



In [5]:
vgg16 = VGG16(299, 299, train_ds, validation_ds)
model = vgg16.model()
# Checking Model Summary
model.summary()

58889256/58889256 [==============================] - 16s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 299, 299, 3)]     0         
                                                                 
 tf.__operators__.getitem (S  (None, 299, 299, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 299, 299, 3)      0         
                                                                 
 vgg16 (Functional)          (None, 9, 9, 512)         14714688  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                             

In [6]:
vgg16.train_model()

AttributeError: 'function' object has no attribute 'fit'